In [1]:
import pandas as pd
import os
from tqdm import tqdm_notebook

In [2]:
'''
Get the lists of files, which were distributed among the students and volunteers,
and later received in a complete form
'''
location = '03_for_students/02_complete_data'
files = os.listdir(location)

location_raw = '03_for_students/01_distribution_data'
files_raw = [item for item in os.listdir(location_raw) if len(item.split('.'))>1 if item.split('.')[1] == 'xlsx']

files_not_received = []

[files_raw.remove(filename) for filename in files_not_received]
print("")

In [5]:
'''
Functions to run checks on received data
'''

'''
Check if the quantity of received files corresponds the quantity of distributed files
'''
def check_document_quantity(files_raw,files):
    if len(files_raw) == len(files):
        print("Перевірка кількості отриманих файлів: OK")
    else:
        print("Перевірка кількості отриманих файлів: Failed")
        if len(files_raw) > len(files):
            for i in set(files_raw).difference(set(files)):
                print("У переліку отриманих файлів не вистачає файлу %s" %i)
        elif len(files_raw) < len(files):
            for i in set(files).difference(set(files_raw)):
                print("У переліку отриманих файлів файл %s є зайвим" %i)

'''
Check if there is a supply schedule for each contract and no excessive supply schedules exist
'''
def check_document_ids(location,current_file):
    main_sheet = pd.read_excel(os.path.join(location,current_file))
    main_sheet_raw = pd.read_excel(os.path.join(location_raw,current_file))
    supply_schedule = pd.read_excel(os.path.join(location,current_file),sheet_name=1)
    initial_rows_only = main_sheet.loc[main_sheet['row_added_manually'] == False]

    unaccounted_ids = get_unaccounted_ids(set(main_sheet['contract_id'].unique()),set(supply_schedule['contract_id'].unique()))
    unaccounted_ids_outer = get_unaccounted_ids(set(main_sheet['contract_id'].unique()),set(main_sheet_raw['contract_id'].unique()))
    unaccounted_ids_rows = []
    
    inner_check(unaccounted_ids,current_file,supply_schedule,main_sheet)
    outer_check(unaccounted_ids_outer,current_file,main_sheet_raw,main_sheet)
    check_rows(unaccounted_ids_rows,current_file,main_sheet_raw,initial_rows_only)

'''
Get set of all contract ids, which were not taken into account
'''
def get_unaccounted_ids(ids_set1,ids_set2):
    all_ids = ids_set1.union(ids_set2)
    unaccounted_ids = all_ids.difference(ids_set2)
    if len(unaccounted_ids) == 0:
        unaccounted_ids = all_ids.difference(ids_set1)
    return unaccounted_ids

'''
Check if contract ids in the main sheet correspond to the supply schedule within the same file
'''
def inner_check(unaccounted_ids,current_file,supply_schedule,main_sheet):
    if len(unaccounted_ids) == 0:
        print("Заповнення графіку поставок для %s: OK" %current_file)
    elif len(main_sheet['contract_id'].unique()) == len(supply_schedule['contract_id'].unique()):
        print("Заповнення графіку поставок для %s: Failed" %current_file)
        print("Перелік contract_ids співпадає лише за довжиною, не співпадають такі ids:")
        append_records(current_file,unaccounted_ids,unaccounted_records)
    elif len(main_sheet['contract_id'].unique()) > len(supply_schedule['contract_id'].unique()):
        print("Заповнення графіку поставок для %s: Failed" %current_file)
        print("У графіку поставок не вистачає таких contract_ids:")
        append_records(current_file,unaccounted_ids,unaccounted_records)
    elif len(main_sheet['contract_id'].unique()) < len(supply_schedule['contract_id'].unique()):
        print("Заповнення графіку поставок для %s: Failed" %current_file)
        print("У графіку поставок є такі зайві contract_ids:")
        append_records(current_file,unaccounted_ids,unaccounted_records)
    else:
        print("Заповнення графіку поставок: невідома помилка при перевірці")
        raise ValueError

'''
Check if the contract ids are the same in the sent and received file
'''
def outer_check(unaccounted_ids_outer,current_file,main_sheet_raw,main_sheet):
    if len(unaccounted_ids_outer) == 0:
        print("Кількість ids у заповненому файлі %s: OK" %current_file)
    elif len(main_sheet['contract_id'].unique()) == len(main_sheet_raw['contract_id'].unique()):
        print("Кількість ids у заповненому файлі %s: Failed" %current_file)
        print("Кількість ids у надісланому і заповненому файлі співпадають лише за довжиною, не співпадають такі ids:")
        append_records(current_file,unaccounted_ids_outer,unaccounted_records_out)
    elif len(main_sheet['contract_id'].unique()) > len(main_sheet_raw['contract_id'].unique()):
        print("Кількість ids у заповненому файлі %s: Failed" %current_file)
        print("У заповненому файлі з'явилися такі contract_ids:")
        append_records(current_file,unaccounted_ids_outer,unaccounted_records_out)
    elif len(main_sheet['contract_id'].unique()) < len(main_sheet_raw['contract_id'].unique()):
        print("Кількість ids у заповненому файлі %s: Failed" %current_file)
        print("У заповненому файлі не вистачає таких contract_ids:")
        append_records(current_file,unaccounted_ids_outer,unaccounted_records_out)
    else:
        print("Кількість ids у заповненому файлі: невідома помилка при перевірці")
        raise ValueError

'''
Check the number of rows in the main sheet of sent and received file
'''
def check_rows(unaccounted_ids_rows,current_file,main_sheet_raw,initial_rows_only):
    if len(main_sheet_raw) == len(initial_rows_only):
        print("Перевірка кількості рядків: OK")
    elif len(main_sheet_raw) != len(initial_rows_only):
        if len(main_sheet_raw) > len(initial_rows_only):
            print("Перевірка кількості рядків: не вистачає рядків")
        elif len(main_sheet_raw) < len(initial_rows_only):
            print("Перевірка кількості рядків: надто багато рядків")
        for contract_id in main_sheet_raw['contract_id'].unique():
            if len(main_sheet_raw.loc[main_sheet_raw['contract_id']==contract_id]) != len(initial_rows_only.loc[initial_rows_only['contract_id']==contract_id]):
                unaccounted_ids_rows.append(contract_id)
        append_records(current_file,unaccounted_ids_rows,unaccounted_records_rows)
    else:
        raise ValueError("Перевірка кількості рядків: сталася невідома помилка")

'''
Append records, which did not pass the checks, to the list
'''
def append_records(current_file,unaccounted_ids,unaccounted_records):
    for unaccounted_id in unaccounted_ids:
        print(unaccounted_id)
        unaccounted_records.append((current_file,unaccounted_id))

In [8]:
'''
Initiate lists of unaccounted records
and run all the checks
'''

unaccounted_records = []
unaccounted_records_out = []
unaccounted_records_rows = []

check_document_quantity(files_raw,files)
for current_file in files:
    check_document_ids(location,current_file)

Перевірка кількості отриманих файлів: OK
Заповнення графіку поставок для addcontr_gas_extract1.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract1.xlsx: OK
Перевірка кількості рядків: OK
Заповнення графіку поставок для addcontr_gas_extract10.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract10.xlsx: OK
Перевірка кількості рядків: OK
Заповнення графіку поставок для addcontr_gas_extract11.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract11.xlsx: OK
Перевірка кількості рядків: OK
Заповнення графіку поставок для addcontr_gas_extract12.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract12.xlsx: OK
Перевірка кількості рядків: OK
Заповнення графіку поставок для addcontr_gas_extract13.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract13.xlsx: OK
Перевірка кількості рядків: OK
Заповнення графіку поставок для addcontr_gas_extract14.xlsx: OK
Кількість ids у заповненому файлі addcontr_gas_extract14.xlsx: OK
Перевірка кількості рядків

In [9]:
'''
Create dataframes for unaccounted records
'''

df_unaccounted_records = pd.DataFrame.from_records(unaccounted_records, columns = ['filename','contract_id'])
df_unaccounted_records.to_csv('03_for_students/03_purgatorium/unaccounted_records.csv', index = False)

df_unaccounted_records_outer = pd.DataFrame.from_records(unaccounted_records_out, columns = ['filename','contract_id'])
df_unaccounted_records_outer.to_csv('03_for_students/03_purgatorium/unaccounted_records_outer.csv', index = False)

df_unaccounted_records_rows = pd.DataFrame.from_records(unaccounted_records_rows, columns = ['filename','contract_id'])
df_unaccounted_records_rows.to_csv('03_for_students/03_purgatorium/unaccounted_records_rows.csv', index = False)

In [10]:
'''
Aggregate data from files received from students and volunteers
into a single dataframe
'''

#УВАГА! цифру у першому рядку потрібно оновлювати вручну
rows_received = 7127

columns_main = ['addcontr_date', 
                'rationale_type', 
                'rationale', 
                'contract_id',
                'date_signed', 
                'contract_end', 
                'tender_id', 
                'procuring_entity',
                'contract_value', 
                'region', 
                'tenderURL', 
                'price_start',
                'price_addcontr', 
                'row_added_manually']

columns_schedule = ['contract_id', 'month', 'year', 'amount_1000m3', 'supply_implied']

main_sheet_all = pd.DataFrame(columns = columns_main)
supply_schedule_all = pd.DataFrame(columns = columns_schedule)

for current_file in tqdm_notebook(files):
    main_sheet = pd.read_excel(os.path.join(location,current_file))
    supply_schedule = pd.read_excel(os.path.join(location,current_file),sheet_name=1)
    
    main_sheet_all = main_sheet_all.append(main_sheet, sort=True)
    supply_schedule_all = supply_schedule_all.append(supply_schedule, sort=True)

main_sheet_all = main_sheet_all[columns_main]
supply_schedule_all = supply_schedule_all[columns_schedule]
    
print(main_sheet_all.shape)
print(supply_schedule_all.shape)
if len(main_sheet_all.loc[main_sheet_all['row_added_manually'] == False]) == rows_received:
    print("Кількість рядків у кінцевому файлі відповідає очікуваній")
else:
    print("Кількість рядків у кінцевому файлі відрізняється від очікуваної")
main_sheet_all.head()


(7199, 14)
(12610, 5)
Кількість рядків у кінцевому файлі відповідає очікуваній


,addcontr_date,rationale_type,rationale,contract_id,date_signed,contract_end,tender_id,procuring_entity,contract_value,region,tenderURL,price_start,price_addcontr,row_added_manually
0,2017-09-25 00:00:00,itemPriceVariation,У звязку зі зміною ціни на ринку природного г...,2c9ec3e3dc274eb3ade989a5e8f0bae2,2017-09-14,2017-12-30 00:00:00,UA-2017-08-04-000060-b,Відділ освіти Верхньодніпровської районної де...,1525673.82,Дніпропетровська область,https://prozorro.gov.ua/tender/UA-2017-08-04-0...,6476.52,7123.53,False
1,2017-10-10 00:00:00,itemPriceVariation,У звязку зі зміною ціни на ринку природного га...,2c9ec3e3dc274eb3ade989a5e8f0bae2,2017-09-14,2017-12-30 00:00:00,UA-2017-08-04-000060-b,Відділ освіти Верхньодніпровської районної де...,1525673.82,Дніпропетровська область,https://prozorro.gov.ua/tender/UA-2017-08-04-0...,6476.52,7835.17,False
2,2017-10-23 00:00:00,itemPriceVariation,У звязку зі зміною ціни на ринку природного га...,2c9ec3e3dc274eb3ade989a5e8f0bae2,2017-09-14,2017-12-30 00:00:00,UA-2017-08-04-000060-b,Відділ освіти Верхньодніпровської районної де...,1525673.82,Дніпропетровська область,https://prozorro.gov.ua/tender/UA-2017-08-04-0...,6476.52,8617.91,False
3,2017-11-02 00:00:00,itemPriceVariation,У звязку зі зміною ціни на ринку природного га...,2c9ec3e3dc274eb3ade989a5e8f0bae2,2017-09-14,2017-12-30 00:00:00,UA-2017-08-04-000060-b,Відділ освіти Верхньодніпровської районної де...,1525673.82,Дніпропетровська область,https://prozorro.gov.ua/tender/UA-2017-08-04-0...,6476.52,9478.84,False
4,2017-11-20 00:00:00,itemPriceVariation,У звязку зі зміною ціни на ринку природного га...,2c9ec3e3dc274eb3ade989a5e8f0bae2,2017-09-14,2017-12-30 00:00:00,UA-2017-08-04-000060-b,Відділ освіти Верхньодніпровської районної де...,1525673.82,Дніпропетровська область,https://prozorro.gov.ua/tender/UA-2017-08-04-0...,6476.52,9600,False


In [11]:
'''
Export dataframe with aggregated data into xlsx
'''
writer = pd.ExcelWriter('04_output/addcontr_all.xlsx')
main_sheet_all.to_excel(writer,'contracts_short',index = False)
supply_schedule_all.to_excel(writer,'schedule',index = False)
writer.save()